In [4]:
import os
import json
import plotly.graph_objects as go
from plotly.subplots import make_subplots
# import pymongo
from tqdm import tqdm
root = "../Dataset/IMDB/"
summary_path = 'IMDB_SUMMARY/SUMMARY_DATA/'
movie_path = 'MOVIES/'

In [6]:
limit = 50000

#### load summary

In [7]:
summary = []

for frm in tqdm(range(1, limit, 250)):
    file_name = '{} - {}.json'.format(frm, frm+250-1)
    with open(root + summary_path + file_name, 'r') as f:
        cur_file = json.load(f)
    summary = summary + cur_file

100%|██████████| 200/200 [00:01<00:00, 182.75it/s]


In [8]:
len(summary)

50000

#### load movies

In [11]:
# os.listdir(root + movie_path + '1 - 250/')
# with open(root + movies + '1 - 250/' + 'tt9612102__exit.json', 'r') as f:
#     movie = json.load(f)
# movie

In [12]:
movie_list = []

for frm in tqdm(range(1, limit, 250)):
    folder_name = '{} - {}/'.format(frm, frm+250-1)
    movie_files = os.listdir(root + movie_path + folder_name)
    for file_name in movie_files:
        file_path = root + movie_path + folder_name + file_name
        with open(file_path, 'r') as f:
            cur_movie = json.load(f)
        movie_list.append(cur_movie)

100%|██████████| 200/200 [00:12<00:00, 15.94it/s]


In [35]:
movie_list[111]

{'@context': 'http://schema.org',
 '@type': 'Movie',
 'url': '/title/tt1853728/',
 'name': 'Django Unchained',
 'image': 'https://m.media-amazon.com/images/M/MV5BMjIyNTQ5NjQ1OV5BMl5BanBnXkFtZTcwODg1MDU4OA@@._V1_.jpg',
 'genre': ['Drama', 'Western'],
 'contentRating': 'R',
 'actor': [{'@type': 'Person',
   'url': '/name/nm0004937/',
   'name': 'Jamie Foxx'},
  {'@type': 'Person', 'url': '/name/nm0910607/', 'name': 'Christoph Waltz'},
  {'@type': 'Person', 'url': '/name/nm0000138/', 'name': 'Leonardo DiCaprio'},
  {'@type': 'Person', 'url': '/name/nm0913488/', 'name': 'Kerry Washington'}],
 'director': {'@type': 'Person',
  'url': '/name/nm0000233/',
  'name': 'Quentin Tarantino'},
 'creator': [{'@type': 'Person',
   'url': '/name/nm0000233/',
   'name': 'Quentin Tarantino'},
  {'@type': 'Organization', 'url': '/company/co0150452/'},
  {'@type': 'Organization', 'url': '/company/co0050868/'}],
 'description': 'Django Unchained is a movie starring Jamie Foxx, Christoph Waltz, and Leonardo 

# @type

In [16]:
type_dict = {}
for movie in movie_list:
    __type = movie['@type']
    if(__type not in type_dict):
        type_dict[__type] = []
    type_dict[__type].append(movie)

In [17]:
for __type in type_dict:
    print(__type, len(type_dict[__type]))

TVSeries 11859
Movie 35785
CreativeWork 1505
VideoGame 573
TVEpisode 11


In [18]:
movie_list = type_dict['Movie']

In [19]:
len(movie_list)

35785

# Release Date - Budget

In [22]:
months =    ['January', 'February', 'March', 'April', 'May', 'June',
            'July', 'August', 'September', 'October', 'November', 'December']
years = []
for year in range(1900, 2025):
    years.append(str(year))

days = []
for day in range(1, 32):
    days.append(str(day))

In [27]:
def getDate(date_str):
    date_str = date_str.split(" ")
    day = None
    month = None
    year = None

    for val in date_str:
        if(day == None and val in days):
            day = val
        if(month == None and val in months):
            month = val
        if(year == None and val in years):
            year = val
    
    return day, month, year

In [28]:
getDate('22 February 2017 (South Korea)')

('22', 'February', '2017')

In [29]:
month_movie = {}
year_movie = {}

for month in months:
    month_movie[month] = []
for year in years:
    year_movie[year] = []

no_info = 0
for movie in movie_list:
    try:
        date_str = movie['details']['Release Date'][0]
        day, month, year = getDate(date_str)
        if(month != None):
            month_movie[month].append(movie)
        if(year != None):
            year_movie[year].append(movie)
    except:
        if('Release Date' not in movie['details']):
            no_info += 1
        else:
            print(movie['details']['Release Date'])

In [31]:
no_info # number of movies that does not have release date available

1071

In [33]:
xrr = []
yrr = []

for month in month_movie:
    print(month, len(month_movie[month]))
    xrr.append(month)
    yrr.append(len(month_movie[month]))

January 2666
February 2746
March 3053
April 2834
May 2630
June 2510
July 2321
August 2740
September 3015
October 3529
November 3019
December 2799


In [34]:
fig = go.Figure([go.Bar(x=xrr, y=yrr)])
fig.show()

In [39]:
xrr = []
yrr = []

for year in year_movie:
    xrr.append(year)
    yrr.append(len(year_movie[year]))

In [40]:
fig = go.Figure([go.Bar(x=xrr, y=yrr)])
fig.show()

## Revenue - Month

In [77]:
def getMoney(txt):
    converter = {
        '$'  : 1, 'USD': 1, 'EUR': 1.22, 'AUD': 0.78,
        'GBP': 1.41, 'NOK': 0.12, 'DKK': 0.16, 'JPY': 0.0092,
        'CAD': 0.83, 'INR': 0.012, 'SEK': 0.12, 'DEM': 0.62,
        'FRF': 0.185, 'RUR': 0.014, 'CNY': 0.16, 'BEF': 1/33.5,
        'MXN': 0.050, 'CZK': 0.048, 'ZAR': 0.071, 'HKD': 0.13,
        'BRL': 0.19, 'IDR': 0.000070, 'ITL': 0.000626628, 'ARS': 0.011,
        'UAH': 0.036, 'NZD': 0.73, 'TRL': 0.12, 'FIM': 0.204585,
        'HUF': 0.0034, 'DOP': 0.018, 'KRW': 0.00089, 'ESP': 1.22, 
        'EGP': 0.064, 'PLN': 0.27, 'SGD': 0.75, 'IQD': 0.00068, 
        'IEP': 0.647361, 'CHF': 1.11, 'ISK': 0.0081, 'TWD': 0.036,
        'THB': 0.032, 'MYR': 0.24, 'PYG': 0.00015, 'NGN': 0.0026, 
        'ILS': 0.30, 'IRR': 0.000024, 'NLG': 0.039, 'BDT': 0.012, 
        'AMD': 0.0019, 'MVR': 0.065, 'SAR': 0.27, 'RON': 0.25,
        'ATS': 0.0883949
    }
    txt = txt.replace(",","")
    mul = 1
    for key in converter:
        if key in txt:
            txt = txt.replace(key, "")
            mul = converter[key]
            break
    money = int(txt)
    return money

In [99]:
sum_budget = []
avg_budget = []
num_movies_arr = []

for month in month_movie:
    lst = month_movie[month]
    num_movies = len(lst)
    no_budget = 0
    budget = 0
    for movie in lst:
        try:
            budget += getMoney(movie['details']['Budget'][0])
        except:
            if('Budget' not in movie['details']):
                no_budget += 1
            else:
                print(movie['details']['Budget'])
                no_budget += 1
    sum_budget.append(budget)
    avg_budget.append(budget/(num_movies-no_budget))
    num_movies_arr.append(num_movies-no_budget)
    print("{} >> total movies {}({} do not have Budget info) -- total Budget: {}, average: {}".format(month, num_movies, no_budget, budget, budget/(num_movies-no_budget)))  

January >> total movies 2666(1628 do not have Budget info) -- total Budget: 41588431281, average: 40065926.08959538
February >> total movies 2746(1714 do not have Budget info) -- total Budget: 40331815785, average: 39081216.84593023
March >> total movies 3053(1881 do not have Budget info) -- total Budget: 55405294786, average: 47274142.30887372
April >> total movies 2834(1724 do not have Budget info) -- total Budget: 64137989182, average: 57781972.23603603
May >> total movies 2630(1626 do not have Budget info) -- total Budget: 29709324583, average: 29590960.74003984
June >> total movies 2510(1456 do not have Budget info) -- total Budget: 55074288726, average: 52252645.85009488
July >> total movies 2321(1325 do not have Budget info) -- total Budget: 48315897442, average: 48509937.19076305
August >> total movies 2740(1592 do not have Budget info) -- total Budget: 247028022044, average: 215181203.87108013
September >> total movies 3015(1848 do not have Budget info) -- total Budget: 674624

In [100]:
num_row, num_col = 3, 1
fig = make_subplots(rows=num_row, cols=num_col)

fig.add_trace(
    go.Bar(x=months, y=num_movies_arr, name="# of movies"),
    1, 1
)

fig.add_trace(
    go.Bar(x=months, y=sum_budget, name="Budget"),
    2, 1
)

fig.add_trace(
    go.Bar(x=months, y=avg_budget, name="Average budget"),
    3, 1
)

fig.update_layout(
    height=900, 
    width=1200,
    title_text="Movie monthwise analysis"
)

In [106]:
sum_budget = []
avg_budget = []
num_movies_arr = []

for year in year_movie:
    lst = year_movie[year]
    num_movies = len(lst)
    no_budget = 0
    budget = 0
    for movie in lst:
        try:
            budget += getMoney(movie['details']['Budget'][0])
        except:
            if('Budget' not in movie['details']):
                no_budget += 1
            else:
                print(movie['details']['Budget'])
                no_budget += 1
    sum_budget.append(budget)
    avg = 0
    if(num_movies-no_budget != 0):
        avg = budget / (num_movies-no_budget)
    avg_budget.append(avg)
    num_movies_arr.append(num_movies-no_budget)
    print("{} >> total movies {}({} do not have Budget info) -- total Budget: {}, average: {}".format(year, num_movies, no_budget, budget, avg))  

1900 >> total movies 0(0 do not have Budget info) -- total Budget: 0, average: 0
1901 >> total movies 0(0 do not have Budget info) -- total Budget: 0, average: 0
1902 >> total movies 1(0 do not have Budget info) -- total Budget: 30000, average: 30000.0
1903 >> total movies 1(0 do not have Budget info) -- total Budget: 150, average: 150.0
1904 >> total movies 0(0 do not have Budget info) -- total Budget: 0, average: 0
1905 >> total movies 0(0 do not have Budget info) -- total Budget: 0, average: 0
1906 >> total movies 0(0 do not have Budget info) -- total Budget: 0, average: 0
1907 >> total movies 0(0 do not have Budget info) -- total Budget: 0, average: 0
1908 >> total movies 0(0 do not have Budget info) -- total Budget: 0, average: 0
1909 >> total movies 0(0 do not have Budget info) -- total Budget: 0, average: 0
1910 >> total movies 0(0 do not have Budget info) -- total Budget: 0, average: 0
1911 >> total movies 1(1 do not have Budget info) -- total Budget: 0, average: 0
1912 >> tota

In [107]:
num_row, num_col = 3, 1
fig = make_subplots(rows=num_row, cols=num_col)

fig.add_trace(
    go.Bar(x=years, y=num_movies_arr, name="# of movies"),
    1, 1
)

fig.add_trace(
    go.Bar(x=years, y=sum_budget, name="Budget"),
    2, 1
)

fig.add_trace(
    go.Bar(x=years, y=avg_budget, name="Average budget"),
    3, 1
)

fig.update_layout(
    height=900, 
    width=1200,
    title_text="Movie yearwise analysis"
)

# Release Date - Cumulative Worldwide Gross

In [115]:
sum_budget = []
avg_budget = []
num_movies_arr = []

attr_key = 'Cumulative Worldwide Gross'

for month in month_movie:
    lst = month_movie[month]
    num_movies = len(lst)
    no_budget = 0
    budget = 0
    for movie in lst:
        try:
            budget += getMoney(movie['details'][attr_key][0])
        except:
            if(attr_key not in movie['details']):
                no_budget += 1
            else:
                print(movie['details'][attr_key])
                no_budget += 1
    sum_budget.append(budget)
    avg_budget.append(budget/(num_movies-no_budget))
    num_movies_arr.append(num_movies-no_budget)
    print("{} >> total movies {}({} do not have Budget info) -- total {}: {}, average: {}".format(month, num_movies, no_budget, attr_key, budget, budget/(num_movies-no_budget)))  

January >> total movies 2666(1336 do not have Budget info) -- total Cumulative Worldwide Gross: 35999274280, average: 27067123.51879699
February >> total movies 2746(1377 do not have Budget info) -- total Cumulative Worldwide Gross: 44458152807, average: 32474910.742878012
March >> total movies 3053(1492 do not have Budget info) -- total Cumulative Worldwide Gross: 47583003420, average: 30482385.27866752
April >> total movies 2834(1389 do not have Budget info) -- total Cumulative Worldwide Gross: 38225991330, average: 26453973.238754325
May >> total movies 2630(1342 do not have Budget info) -- total Cumulative Worldwide Gross: 71304878092, average: 55360930.19565217
June >> total movies 2510(1281 do not have Budget info) -- total Cumulative Worldwide Gross: 79853594609, average: 64974446.38649309
July >> total movies 2321(1120 do not have Budget info) -- total Cumulative Worldwide Gross: 75957910873, average: 63245554.43213988
August >> total movies 2740(1216 do not have Budget info) -

In [116]:
num_row, num_col = 3, 1
fig = make_subplots(rows=num_row, cols=num_col)

fig.add_trace(
    go.Bar(x=months, y=num_movies_arr, name="# of movies"),
    1, 1
)

fig.add_trace(
    go.Bar(x=months, y=sum_budget, name="Revenue"),
    2, 1
)

fig.add_trace(
    go.Bar(x=months, y=avg_budget, name="Average Revenue"),
    3, 1
)

fig.update_layout(
    height=900, 
    width=800,
    title_text="Movie monthwise revenue"
)

In [110]:
sum_budget = []
avg_budget = []
num_movies_arr = []

attr_key = 'Cumulative Worldwide Gross'

for year in year_movie:
    lst = year_movie[year]
    num_movies = len(lst)
    no_budget = 0
    budget = 0
    for movie in lst:
        try:
            budget += getMoney(movie['details'][attr_key][0])
        except:
            if(attr_key not in movie['details']):
                no_budget += 1
            else:
                print(movie['details'][attr_key])
                no_budget += 1
    sum_budget.append(budget)
    avg = 0
    if(num_movies-no_budget != 0):
        avg = budget / (num_movies-no_budget)
    avg_budget.append(avg)
    num_movies_arr.append(num_movies-no_budget)
    print("{} >> total movies {}({} do not have {} info) -- total {}: {}, average: {}".format(year, num_movies, no_budget, attr_key, attr_key, budget, avg))  

1900 >> total movies 0(0 do not have Cumulative Worldwide Gross info) -- total Cumulative Worldwide Gross: 0, average: 0
1901 >> total movies 0(0 do not have Cumulative Worldwide Gross info) -- total Cumulative Worldwide Gross: 0, average: 0
1902 >> total movies 1(1 do not have Cumulative Worldwide Gross info) -- total Cumulative Worldwide Gross: 0, average: 0
1903 >> total movies 1(1 do not have Cumulative Worldwide Gross info) -- total Cumulative Worldwide Gross: 0, average: 0
1904 >> total movies 0(0 do not have Cumulative Worldwide Gross info) -- total Cumulative Worldwide Gross: 0, average: 0
1905 >> total movies 0(0 do not have Cumulative Worldwide Gross info) -- total Cumulative Worldwide Gross: 0, average: 0
1906 >> total movies 0(0 do not have Cumulative Worldwide Gross info) -- total Cumulative Worldwide Gross: 0, average: 0
1907 >> total movies 0(0 do not have Cumulative Worldwide Gross info) -- total Cumulative Worldwide Gross: 0, average: 0
1908 >> total movies 0(0 do not 

In [112]:
num_row, num_col = 3, 1
fig = make_subplots(rows=num_row, cols=num_col)

fig.add_trace(
    go.Bar(x=years, y=num_movies_arr, name="# of movies"),
    1, 1
)

fig.add_trace(
    go.Bar(x=years, y=sum_budget, name="Revenue"),
    2, 1
)

fig.add_trace(
    go.Bar(x=years, y=avg_budget, name="Average Revenue"),
    3, 1
)

fig.update_layout(
    height=900, 
    width=1200,
    title_text="Movie yearwise revenue"
)